In [1]:
import sys
sys.path.append("../extraction_core/page_selection")
sys.path.append("../extraction_core/extraction")
sys.path.append("../extraction_core/")

In [2]:
%env MLFLOW_TRACKING_URI=https://projet-extraction-tableaux-818146.user.lab.sspcloud.fr
%env MLFLOW_S3_ENDPOINT_URL=https://minio.lab.sspcloud.fr

env: MLFLOW_TRACKING_URI=https://projet-extraction-tableaux-818146.user.lab.sspcloud.fr
env: MLFLOW_S3_ENDPOINT_URL=https://minio.lab.sspcloud.fr


In [3]:
import os
os.environ["MLFLOW_S3_ENDPOINT_URL"]

'https://minio.lab.sspcloud.fr'

In [4]:
os.environ["MLFLOW_TRACKING_URI"]

'https://projet-extraction-tableaux-818146.user.lab.sspcloud.fr'

In [5]:
import hvac
client = hvac.Client(
    url="https://vault.lab.sspcloud.fr", token=os.environ["VAULT_TOKEN"]
)

secret = os.environ["VAULT_MOUNT"] + os.environ["VAULT_TOP_DIR"] + "/s3_creds"
mount_point, secret_path = secret.split("/", 1)
secret_dict = client.secrets.kv.read_secret_version(
    path=secret_path, mount_point=mount_point
)

os.environ["AWS_ACCESS_KEY_ID"] = secret_dict["data"]["data"][
    "ACCESS_KEY"
]
os.environ["AWS_SECRET_ACCESS_KEY"] = secret_dict["data"]["data"][
    "SECRET_KEY"
]
try:
    del os.environ["AWS_SESSION_TOKEN"]
except KeyError:
    pass

In [8]:
from page_selection.page_selector import PageSelector
import mlflow
from extraction.data import fs


model_name = "page_selection"
clf = mlflow.pyfunc.load_model(f"models:/{model_name}/16")
page_selector = PageSelector(
    clf=clf
)

orange_list = [
    "306769688",
    "307299248",
    "345039416",
    "379984891",
    "380129866",
    "384518114",
    "409759156",
    "430107359",
    "432668432",
    "440186740",
    "440419240",
    "500413505",
    "500440813",
    "501614572" 
]

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
import tempfile

for siren in orange_list:
    print("---")
    print(siren)
    bmp_path = "projet-extraction-tableaux/orange/bmp/" + siren + ".bmp"
    pdf_path = (
        "projet-extraction-tableaux/orange/" + siren + ".pdf"
    )
    file_name = siren + ".bmp"
    try:
        page_image = page_selector.get_page(pdf_path, dpi=300)
        with tempfile.TemporaryDirectory() as tmpdirname:
            page_image.save(tmpdirname + file_name, 'BMP')
            fs.put_file(tmpdirname + file_name, bmp_path)
    except ValueError:
        continue

---
306769688
---
307299248
---
345039416
---
379984891
---
380129866
---
384518114
---
409759156
---
430107359
---
432668432
---
440186740
---
440419240
---
500413505
---
500440813
---
501614572


On a récupéré 11 fichiers sur 14, manquants : 440186740, 380129866 et 306769688

Sur ceux qu'on a, pbs ? Non. Missing ?

In [50]:
siren = "306769688"
pdf_path = (
    "projet-extraction-tableaux/orange/" + siren + ".pdf"
)
page_image = page_selector.get_page(pdf_path, dpi=300)

ValueError: Pas de tableau filiales et participations détecté.

In [51]:
from PIL import Image
from page_selection.utils import clean_page_content, extract_document_content, load_pdf
import mlflow

page_list = extract_document_content(
    pdf_path, True
)

In [52]:
page_list[19]

"Exprimé en €\nFiliales et participations\nR\né\ns\ne\nr\nv\nQuote-\npart du \ncapital \ndétenu\nCapital social\nDevises\nDate des \nétats \nfinanciers\nBrute\nNette\nFair Value*\nA.    Renseignements détaillés\nOrange Capital Management\n3 342 864\n3 342 864\n100%\n2 346 000\n-3 382\nEUR\n31-déc.-21\nOrange Digital Ventures Support\n68 200 000\n68 200 000\n100%\n68 200 000\n-4 485 166\nEUR\n31-déc.-21\nOrange Digital Ventures Africa\n152\n152\n100%\n100 000\n-142 304 077\nXOF\n31-déc.-21\nOVCI\n2 377 768\n2 377 768\n70%\n34 140 050\n-593 741\nRMB\n31-déc.-21\nOrange Technologies Investissements\n26 611 500\n7 654 607\n100%\n619 057\n6 926 639\nEUR\n31-déc.-21\n2 062 189\nOrange Ventures MEA SEED Investments Fund1\n1 510 578\n1 400 599\n100%\n1 510 000\n-119 629\nEUR\n31-déc.-21\nOrange Ventures\n8 582\n8 582\n100%\n8 500\n2 045 049\nEUR\n31-déc.-21\nOrange Ventures MEA Champion Fund1\n10 010 000\n8 812 215\n100%\n10 010 000\n-1 271 190\nEUR\n31-déc.-21\nOrange Ventures Global Champion

In [53]:
doc = load_pdf(pdf_path, True)
page_list = []

for page in doc:
    page_content = page.get_text()
    if not page_content:
        page_list.append("vide")
    else:
        page_list.append(page_content)

In [54]:
clean_page_list = []
for page in page_list:
    clean_page_list.append(clean_page_content(page))

clean_page_list[19]

'exprim filial particip r e e r v quot part capital detenu capital social devis dat etat financi brut net fair valu a renseign detaill orang capital manag 3 342 864 3 342 864 100 2 346 000 3 382 eur 31 dec 21 orang digital ventur support 68 200 000 68 200 000 100 68 200 000 4 485 166 eur 31 dec 21 orang digital ventur afric 152 152 100 100 000 142 304 077 xof 31 dec 21 ovci 2 377 768 2 377 768 70 34 140 050 593 741 rmb 31 dec 21 orang technolog invest 26 611 500 7 654 607 100 619 057 6 926 639 eur 31 dec 21 2 062 189 orang ventur me seed investment fund1 1 510 578 1 400 599 100 1 510 000 119 629 eur 31 dec 21 orang ventur 8 582 8 582 100 8 500 2 045 049 eur 31 dec 21 orang ventur me champion fund1 10 010 000 8 812 215 100 10 010 000 1 271 190 eur 31 dec 21 orang ventur global champion world fund1 11 510 000 9 519 183 100 11 510 000 2 228 183 eur 31 dec 21 orang ventur global champion europ fund1 55 010 000 53 547 438 100 55 010 000 1 989 362 eur 31 dec 21 orang ventur impact investment

In [55]:
model_output = clf.predict(clean_page_list)
predictions = model_output["predictions"]
probas = model_output["probas"]

In [56]:
predictions

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
Name: predictions, dtype: object

In [57]:
probas

0     0.974831
1     1.000010
2     0.997894
3     1.000010
4     1.000010
5     1.000010
6     0.997209
7     1.000010
8     0.998200
9     0.999491
10    1.000010
11    1.000010
12    0.995105
13    0.994455
14    0.999099
15    1.000010
16    0.996737
17    1.000010
18    0.996065
19    0.651365
20    1.000010
21    0.999566
22    1.000010
23    1.000010
24    0.997687
25    1.000010
26    1.000010
27    1.000010
28    1.000010
29    1.000010
30    1.000010
31    1.000010
32    0.997613
33    0.997029
34    1.000010
35    0.998726
Name: probas, dtype: float64

Export à la main

In [58]:
doc

Document('tmp_pdf.pdf')

In [61]:
pix = doc[19].get_pixmap(dpi=300)
mode = "RGBA" if pix.alpha else "RGB"
page_image = Image.frombytes(mode, [pix.width, pix.height], pix.samples)

bmp_path = "projet-extraction-tableaux/orange/bmp/" + siren + ".bmp"
file_name = siren + ".bmp"
with tempfile.TemporaryDirectory() as tmpdirname:
    page_image.save(tmpdirname + file_name, 'BMP')
    fs.put_file(tmpdirname + file_name, bmp_path)

Ok erreur sur celui là, on pourrait enrichir le jeu d'apprentissage avec pe ?

In [62]:
siren = "380129866"
pdf_path = (
    "projet-extraction-tableaux/orange/" + siren + ".pdf"
)

page_list = extract_document_content(
    pdf_path, True
)

In [63]:
page_list

["RGS : NANTERRE Code grette : 9201 Documents comptables REGISTRE DU COMMERCE ET DES SOCIETES Le greffier du tribunal de commerce de NANTERRE atteste l'exactitude des informations transmises ci-après Mature du document : Documents comptables (B-Gi Numéro de gestion : 2001 B 00484 Numéro SIREN : 380 129 866   Nom ou dénomination : Orange Ce dépôt a été enregistré le 31/05/2022 sous le numéro de dépôt 14379",
 'ORANGE S.A. Société anonyme 111, Quai du Président Roosevelt 92130 Issy-les-Moulineaux   Rapport des commissaires aux comptes sur les comptes consolidés Exercice clos le 31 décembre 2021 AU,',
 "kPhG KPMG S.A. Tour EQHO 2 Avenue Gambetta - CS 60055 92066 Paris La Défense Cedex France Société de Commissariat aux Comptes inscrite à la Compagnie Régionale de Versailles et du Centre ORANGE S.A. Société anonyme 111, Quai du Président Roosevelt 92130 issy-les-Moulineaux   Rapport des commissaires aux comptes sur les comptes consolidés Exercice clos le 31 décembre 2021   À l’Assemblée Gé

In [41]:
matching = [(i+1, s) for i, s in enumerate(page_list) if "filiales" in s]

In [42]:
matching

[(20,
  "Tableau des variations des capitaux propres consolidés                                    {en millions d'euros) Note Attribuables aux propriétaires Attribuables aux Total de la société mère participations capitaux ne donnant pas le contrôle propres Nombre Capital Prime Titres Réserves Autres Total Réserves Autres Total d'actions social d'émi-  subor- éléments éléments émises ssion et donnés du résultat du résultat réserve global global légale Solde au 1er janvier 2019 (2060) (571) 30671 ___2357 223 2680 33251 Effet de l'application de la jon d'IFRS 1 _ | D LI 114 après retraitementft 2 660 056 598 10 640 16 859 _ 5803 (1 946 (571) 30785 2357 223_2 580 33 365 Résultat global consolidé{f =: = = Ê 3004 104__3109 218 11__230 3 339 Rémunération en actions 6.1 - - - - 52 - 52 3 - 3 55 Rachat d'actions propres 15.2 - - - - (34) - (34) - - - (34) Distribution de dividendes 15.3 - - - - (1857) - (1857) (248) - (248) (2 105) Emissions et rachats de titres subordonnés 15.4 - - - 0 (81) -

Pas de tableau là, on a peut-être le mauvais doc ?

In [64]:
pdf_path = (
    "test.pdf"
)

page_list = extract_document_content(
    pdf_path, False
)

In [65]:
matching = [(i+1, s) for i, s in enumerate(page_list) if "filiales" in s]
matching

[(17,
  "Tableau des flux de trésorerie       (en millions d'euros) Note 2021 2020 Flux de trésorerie lié à l'activité Résultat de l'exercice (1 905) 2 387 Eléments non monétaires sans incidence sur la trésorerie Mali sur rachat d'actions propres 168 - Amortissements des immobilisations incorporelles et corporelles ( 3 565 3 656 Moins-values (plus-values) sur cession d'actifs (317) (674) Variations des provisions et autres éléments non monétaires © 3 144 2774 Variations du besoin en fonds de roulement Diminution (augmentation) des stocks (72) 107 Diminution (augmentation) des créances clients 139 (531) Diminution (augmentation) des autres créances et charges constatées d'avance 4.3 et4.5 85 (124) Augmentation (diminution) des dettes fournisseurs (hors immobilisations) 285 99 Augmentation (diminution) des autres dettes et produits constatés d'avance 5.6 et 5.7 (225) 192 Variation des écarts de conversion 4.6 105 (142) Flux net de trésorerie généré par l'activité ( 4972 7 744 Flux de tré

Il faut potentiellement quand plusieurs bilans choisir celui de type S ?

In [66]:
clean_page_list = []
for page in page_list:
    clean_page_list.append(clean_page_content(page))

model_output = clf.predict(clean_page_list)
predictions = model_output["predictions"]
probas = model_output["probas"]
std_probas = probas.copy()
for idx, prediction in enumerate(predictions):
    if str(prediction) == "0":
        std_probas[idx] = 1 - probas[idx]

In [67]:
std_probas.idxmax()

25

In [68]:
siren

'380129866'

In [ ]:
doc = load_pdf(pdf_path, False)

pix = doc[25].get_pixmap(dpi=300)
mode = "RGBA" if pix.alpha else "RGB"
page_image = Image.frombytes(mode, [pix.width, pix.height], pix.samples)

bmp_path = "projet-extraction-tableaux/orange/bmp/" + siren + ".bmp"
file_name = siren + ".bmp"
with tempfile.TemporaryDirectory() as tmpdirname:
    page_image.save(tmpdirname + file_name, 'BMP')
    fs.put_file(tmpdirname + file_name, bmp_path)

Ajout BMP à la main